In [ ]:
#This code needs a high memory machine to run

In [4]:
import numpy as np
import pandas as pd
import scanpy as sc
import pymn
import anndata as ad
import time
import os
from pyprojroot import here
import resource

In [5]:
base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"

In [9]:
#from braincelldata.org
adata_macosko = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/macosko/from_google_drive/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.h5ad")

In [11]:
adata_macosko.obs['cell.type'] = adata_macosko.obs['ClusterNm'].astype(str)
adata_macosko.obs['study_id'] = "Macosko"
adata_macosko.obs.index = adata_macosko.obs.index.to_numpy(dtype="str")

In [12]:
print("Number of Macosko cell types: " + str(len(set(adata_macosko.obs["cell.type"]))))


Number of Macosko cell types: 5030


In [13]:
##########
### Zeng
##########

In [14]:
adata_Zeng = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.h5ad")

In [15]:
adata_Zeng.obs.groupby('method').count()

/tmp/ipykernel_2454542/781402409.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata_Zeng.obs.groupby('method').count()


,library_prep,gene.counts.0,doublet_score,roi,umi.counts,sex,external_donor_name,age,medical_conditions,cl
method,,,,,,,,,,
10Xv2,1700767,1700767,1700767,1700767,1700767,1700767,1700767,1700767,1700767,1700767
10Xv3,2342082,2342082,2342082,2342082,2342082,2342082,2342082,2342082,2342082,2342082
10xMulti,1687,1687,1687,1687,1687,1687,1687,1687,1687,1687


In [18]:
base_data_folder + "whole_mouse_brain/zeng/from_aws/"

'/vault/lfrench/mouse_brain_cluster_replicability/data/whole_mouse_brain/zeng/from_aws/'

In [19]:
Zeng_cluster_info_AWS = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep = '\t')

In [20]:
#set study ID here to prevent view as actual memory grabs 
Zeng_cluster_info_AWS['study_id'] = "Zeng"
Zeng_cluster_info_AWS['cell.type'] = Zeng_cluster_info_AWS['cl'].astype(str)

In [21]:
#set it to be strings
adata_Zeng.obs['cl'] = adata_Zeng.obs['cl'].astype(str)
Zeng_cluster_info_AWS['cl'] = Zeng_cluster_info_AWS['cl'].astype(str)

In [22]:
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs = adata_Zeng.obs.merge(Zeng_cluster_info_AWS, how="left", on='cl')

In [23]:
#use ensmbl ID as index
adata_Zeng.var = adata_Zeng.var.set_index("gene_identifier")

In [25]:
#cells lost
adata_Zeng.obs['cluster_id'].isna().sum() #none it seems

0

In [26]:
#LQ or low quality cells
sum(adata_Zeng.obs['subclass_label'] == "LQ")

1560

In [27]:
#remove #LQ 
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs["filter_out"] = adata_Zeng.obs['subclass_label'] == "LQ"
adata_Zeng = adata_Zeng[~adata_Zeng.obs["filter_out"],:]
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)

/tmp/ipykernel_2454542/3891690664.py:5: ImplicitModificationWarning: Trying to modify index of attribute `.obs` of view, initializing view as actual.
  adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)


ValueError: To copy an AnnData object in backed mode, pass a filename: `.copy(filename='myfilename.h5ad')`. To load the object into memory, use `.to_memory()`.

In [28]:
print("Number of Zeng cell types: " + str(len(set(adata_Zeng.obs["cell.type"]))))

Number of Zeng cell types: 5322


In [ ]:
merged=ad.concat([adata_Zeng, adata_macosko], join="inner")

In [ ]:
#free memory
adata_Zeng = None
adata_macosko = None

In [ ]:
pymn.variableGenes(merged, study_col='study_id')

In [ ]:
sum(merged.var.highly_variable)

In [ ]:
#subset to highly variable genes to free up memory
merged = merged[:, merged.var.highly_variable]

In [ ]:
merged.obs['cell.type'] = merged.obs['cell.type'].to_numpy(dtype="str")
merged.obs['study_id'] = merged.obs['study_id'].to_numpy(dtype="str")
merged.obs.index = merged.obs.index.to_numpy(dtype="str")

In [ ]:
#use numpy types instead of pandas
merged.var.highly_variable = merged.var.highly_variable.to_numpy(dtype="bool")
merged.var.index = merged.var.index.to_numpy(dtype="str")

In [ ]:
#make cell indices unique
merged.obs_names_make_unique()

In [ ]:
#save
merged.write(base_data_folder + '/whole_mouse_brain/processed/merged_Zeng_AWS.Oct2023.h5ad')